In [1]:
import tensorflow as tf
import os

SAVE_PATH = './save'
MODEL_NAME = 'test'
VERSION = 1
SERVE_PATH = './serve/{}/{}'.format(MODEL_NAME, VERSION)

In [2]:
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)

In [3]:
tf.reset_default_graph()

In [4]:
with tf.Session() as sess:
    # Import the saved graph
    saver = tf.train.import_meta_graph(checkpoint + '.meta')
    # get the graph for this session
    graph = tf.get_default_graph()
    sess.run(tf.global_variables_initializer())
    # get the tensor that we need
    inputs = graph.get_tensor_by_name('inputs:0')
    predictions = graph.get_tensor_by_name('prediction/Sigmoid:0')

    # Create tensor info
    model_input = tf.saved_model.utils.build_tensor_info(inputs)
    model_output = tf.saved_model.utils.build_tensor_info(predictions)

    # Build signature definition
    signature_definition = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={'inputs':model_input},
    outputs = {'outputs': model_output},
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)

    # Prediction API constants

    # Predict inputs.
    PREDICT_INPUTS = 'inputs'

    # Prediction method name used in a SignatureDef.
    PREDICT_METHOD_NAME = 'tensorflow/serving/predict'

    #Precict outputs.
    PREDICT_OUTPUTS = 'outputs'

    builder = tf.saved_model.builder.SavedModelBuilder(SERVE_PATH)

    builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_definition
    })

    # save the model so that we can serve it with amodel server
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./serve/test/1/saved_model.pb
